In [7]:
#%save Kernel_BE.py 4

    

In [1]:
##### coding: utf-8
import bz2 
import lzma
import gzip 
from datetime import datetime, timedelta
from random import randint
import numpy as np

from elasticsearch import Elasticsearch 
from elasticsearch.connection import Urllib3HttpConnection
from elasticsearch.helpers import bulk

class rien () :
    def compress (self, x) :
        return x
    def decompress (self, x) :
        return x



class Kernel () :
    def __init__ ( self, dico, dispersion = 1000000) :
        '''
        
                  createur = "generic", 
                  date_emetteur = datetime.now(),
                  http_auth = None,
                  timeout=10,
                  host = "localhost", 
                  port = 9200 ,
                  zipChoisi = 'bz2',
                  index_log_error = "trace1",
                  index_log_warning = "trace2",
                  index_log_trace = "trace3",
                  index_system = "systeme",
                  isPurge_existing_index_log = True,
                  extra_elasticsearch_args = None,
                  trace = False,
                  FF
        
        ######### centralise les I/O #########
        en entree 
        le createur du process (esemble de taches) 
        la date de lancement par defaut ce jour
        
        ensuite
        pour elastic : host, port
        pour le zip : le choix par nom if none on ne fait rien,
        
        ensuite pour debug la trace (log elastic)
        '''
        self.dispersion = int(dispersion)
        
        self.createur = dico ["createur"]
        date_emetteur = dico ["date_emetteur"]
        http_auth = dico ["hfrom Entree_sortie_lock import Entree_sortie_lockttp_auth"]
        timeout = dico ["timeout"]
        host = dico ["host"]
        port = dico ["port"]
        zipChoisi = dico ["zipChoisi"]
        index_log_error = dico ["index_log_error"]
        index_log_warning = dico ["index_log_warning"]
        index_log_trace = dico ["index_log_trace"]
        index_system = dico ["index_system"]
        isPurge_existing_index_log = dico ["isPurge_existing_index_log"]
        extra_elasticsearch_args = dico ["extra_elasticsearch_args"]
        trace = dico ["trace"]
        #modif ##########################################"
        self.ID_reference_base = dico ["ID_reference_base"]
        
        self.lecture_evenements = "../data/generation/evenements1.txt" # modif client dico ['lecture_evenements']
        
        
        self.date_emetteur = date_emetteur
        self.trace = trace
        
        self.index_log_error = index_log_error
        self.index_log_warning = index_log_warning
        self.index_log_trace = index_log_trace
        self.isPurge_existing_index_log = isPurge_existing_index_log
        
        
        
        self.host = host
        self.port = port
        self.http_auth = http_auth
        self.timeout = timeout
        
        if extra_elasticsearch_args is None:
            extra_elasticsearch_args = {}
        self.extra_elasticsearch_args = extra_elasticsearch_args
         
        
        self.es = Elasticsearch(
                        connection_class = Urllib3HttpConnection, # fonction externe
                        host=self.host,
                        port=self.port,
                        http_auth=self.http_auth, # donne pour authentification
                        timeout=self.timeout,
                        **self.extra_elasticsearch_args,
                        )
        
        
        # init du system
        self.index_system = index_system
        index = index_system
        if isPurge_existing_index_log or not self.es.indices.exists(index=index):
            self.delete_index (index)
            self.create_index (index)
            # init date pour recuperer les logsdate = datetime.datetime.now() mis dans creationDoc
            doc = self.creationDoc ("systeme", "kernel", "init",  "pour recuperer les logs",)
            self.index_doc (doc, index, _id = 1 )
        
        # intialisation compression de données voir pb "byte to string et inversement"        
        self.dicoZip = {'bz2' : bz2,
                  'lzma' : lzma,
                   'gzip' : gzip,
                        }
        self.initZip (zipChoisi = zipChoisi)
        
        
        
               
        # pour la lemmatization       
        
        self.liste_type = ['NOM', 'AUX', 'VER', 'ADV', 'PRE', 'ADJ', 'ONO', 'CON',
                  'ART:def', 'ADJ:ind', 'PRO:ind', 'PRO:int', 'PRO:rel',
                  'ADJ:num', 'PRO:dem', 'ADJ:dem', 'PRO:per', 'ART:ind',
                  'LIA', 'PRO:pos', 'ADJ:pos', '', 'ADJ:int']
        
        self.index_regroupement = "regroupement"
        
        self.settings_regroupement = { "settings" : {
                                      'index.mapping.total_fields.limit':100000,
                                     }
        }
        
        self.dico_settings = {'regroupement' : self.settings_regroupement ,
                             }
                              
                              
        
        self.structure_du_lexique = ["1_ortho" , 
                                     "3_lemme" , 
                                     "4_cgram" , 
                                     "5_genre" , 
                                     "6_nombre" , 
                                     "7_freqlemfilms2" , 
                                     "8_freqlemlivres" , 
                                     "9_freqfilms2" , 
                                     "10_freqlivres" , 
                                     "11_infover" , 
                                     "12_nbhomogr" , 
                                     "13_nbhomoph" , 
                                     "14_islem" , 
                                     "15_nblettres" , 
                                     "19_voisorth" , 
                                     "21_puorth" , 
                                     "22_puphon" , 
                                     "29_cgramortho" , 
                                     "30_deflem" , 
                                     "31_defobs" , 
                                     "32_old20" , 
                                     "33_pld20" , 
                                     "35_nbmorph" , 
                                    ]
        

        
        self.debut_fichier_regroupement = "../../../data/Lexique383_group" # a voir
        
    @property
    def raise_on_error(self):
        """
        Renvoyer False pout permettre à l'appelant de gérer le log d'erreur
        dans le cas du bulk
        """
        return False
    
    @property
    def mapping_log(self,) :
        """
        Dictionary with custom mapping or `None`.
        probleme avec Elastic sur le mapping ??????????
        """
        return None
        
    @property
    def mapping(self):
        """
        Dictionary with custom mapping or `None`.
        probleme avec Elastic sur le mapping ??????????
        """
        mapping =  {
                        "properties": {
                            "ID": {
                                "type": "keyword" # formerly "string"
                            },
                            "ID_reference": {
                                "type": "keyword"
                            },
                            "date_evenement": {
                                "type": "keyword"
                            },
                            "nom_variable": {
                                "type": "keyword"
                            },
                            "valeur" : {
                                "type" : "keyword"
                            }
                        }
                    }
                
        
        return mapping
    
    @property
    def mapping_vecteur(self):
        """
        Dictionary with custom mapping or `None`.
        
        """
        mapping =  {
                        "properties": {
                            "ID": {
                                "type": "integer" 
                            },
                            "ID_random": {
                                "type": "integer"
                            },
                            "vecteur": {
                                "type": "float"
                            },
                            "paragraphe": {
                                "type": "keyword"
                            }
                        }
                    }
                
        
        return mapping
    
    def liste2array(self, data,dtype = np.int32) :
        return np.array(data, dtype = dtype)
    
    def array2liste (self,array,):
        return array.tolist()
        
    
    
    
       
    def compression (self, data) :
        return self.zip.compress (data)
    
    def decompression (self, dataCompresse) :
        return self.zip.decompress (dataCompresse)
    
    def initZip (self, zipChoisi = None ) :
        if zipChoisi is None :
            self.zip = rien()
        else:
            self.zip = self.dicoZip[zipChoisi]
            
    def create_index(self, index):
        """
        creation de l' index si il n'existe pas.
        """
        
        if not self.es.indices.exists(index=index):
            self.es.indices.create(index=index)
            
    def delete_index(self, index):
        """
        Supprime l'index, si il existe.
        """
        
        if self.es.indices.exists(index=  index):
            self.es.indices.delete(index = index)
            
    def _docs_vecteur (self, docIterator, index) :
        dico = {'_index' : index, }
        for doc in docIterator :
            doc ['ID_random'] = randint (0,self.dispersion)
            dico ['_source'] = doc
            yield dico
        return
            
    
            
    def _docs (self, docIterator, index) :
        dico = {'_index' : index,}
        for doc in docIterator :
            dico ['_source'] = doc
            yield dico
        return
    
    def bulk_vecteur (self,
                      docs,
                      index,
                      isPurge_existing_index = False,
                      chunk_size = 2000,
                     ):
        """
        
        docs est un iterateur comme son nom ne l'indique pas
        
        """
        
        
        if isPurge_existing_index:
            self.delete_index(index) # si existe detruit
            
        self.create_index(index) # si existe ne fait rien
        
        if not self.mapping_vecteur is None and isPurge_existing_index:
                self.es.indices.put_mapping(index = index, body = self.mapping_vecteur)
        
        self.es.indices.put_settings({"index": {"refresh_interval": "-1"}},
                                    index= index)

        bulk(self.es, self._docs_vecteur (docs, index) , chunk_size = chunk_size,
                  raise_on_error=self.raise_on_error) # en cas d'erreur on renvoie False

        self.es.indices.put_settings({"index": {"refresh_interval": "1s"}},
                                         index = index)
        self.es.indices.refresh()
        
        return True # tout est OK
    
    
       
    def bulk (self, docs, index,
              isPurge_existing_index = False,
              chunk_size = 2000,
             ):
        """
        
        docs est un iterateur comme son nom ne l'indique pas
        
        """
        mapping = self.mapping
        #print ("mapping =", mapping)
        
        if isPurge_existing_index:
            self.delete_index(index) # si existe detruit
            
        self.create_index(index) # si existe ne fait rien
        
        if not mapping is None and isPurge_existing_index:
            #print ("mapping =", mapping)
            self.es.indices.put_mapping(index = index, body = mapping)
        
        self.es.indices.put_settings({"index": {"refresh_interval": "-1"}},
                                    index= index)

        bulk(self.es, self._docs (docs, index) , chunk_size = chunk_size,
                  raise_on_error=self.raise_on_error) # en cas d'erreur on renvoie False

        self.es.indices.put_settings({"index": {"refresh_interval": "1s"}},
                                         index = index)
        self.es.indices.refresh()
        
        return True # tout est OK 
    
    def count (self, index) :
        if not self.es.indices.exists(index=  index):
            return 0
        
        self.es.indices.refresh(index)
        r = self.es.cat.count(index, params={"format": "json"})
        return int (r [0] ['count'])
    
    # exemple de query sur les mots pour trouver les lemmes
    def search_mot (self, index, mot, zone) :
        self.es.indices.refresh (index)
        query = {'query' : {'match' : {zone : mot}}}
        #print ("query =", query )
        res= self.es.search (index= index, body = query )
        #res = self.elastic.search(index=index , body={"query": {"match_all": {}}})
        #print ([hit['_source'] for hit in res ['hits'] ['hits']])
        return [hit['_source'] for hit in res ['hits'] ['hits']]
    
    def creationDoc (self,origine, auteur, etape,  message,) :
               
        return {"origine": origine,
               "auteur" : auteur,
                "etape" : etape,
                  "date" : datetime.now(),
                   "message" : message, }
        
    
    def log_error (self, auteur, etape,  message,) :
        index = self.index_log_error
        doc = self.creationDoc ("error", auteur, etape,  message,)
        return self.index_doc (doc, index,)
    
    def log_warning (self, auteur, etape,  message,) :
        index = self.index_log_warning
        doc = self.creationDoc ("warning", auteur, etape,  message,)
        return self.index_doc(doc, index,  )
        
    def log_trace (self, auteur, etape, message,  messagesStop = False) :
        index = self.index_log_trace
        doc = self.creationDoc ("trace", auteur, etape,  message,)
        return self.index_doc (doc, index)
        
    def index_doc (self,doc, index, _id = None):
        # on index l'erreur
        if _id is None :
            res = self.es.index (index = index, body = doc)
        else :
            res = self.es.index (index = index, id = _id, body = doc)
        
        self.es.indices.refresh()
        #print ( "dans ecriture erreur res =", res['result'])
        return True # tout est OK
    
    def get_date_system (self,) :
        res = self.es.get(index=self.index_system, id=1)
        doc = res['_source'] 
        return  doc ['date']

    
    
    def get_logs_error (self,) :
        return self.get_logs(self.index_log_error)
    def get_logs_warning (self,) :
        return self.get_logs(self.index_log_warning)
    def get_logs_trace(self,) :
        return self.get_logs(self.index_log_trace)
    
    def get_logs (self, index, size = 10000) :
        
        self.es.indices.refresh (index)
        
        date_system = self.get_date_system ()
        query = {'query' : {'range' : {"date": {"gte" : date_system}}} }
        #print ("query =", query )
        res= self.es.search (index= index, body = query,  size = size )
        #res = self.elastic.search(index=index , body={"query": {"match_all": {}}})
        #print ([hit['_source'] for hit in res ['hits'] ['hits']])
        return [hit['_source'] for hit in res ['hits'] ['hits']]
    
    def calculRandom_vecteur (self, proportion, index,) :
        """
        apres avoir randomiser la variable IDrandom (bulk_vecteur) entre 0 et self.dispersion
        on calcul en fonction de la proportioon voulu des bornes inferieure et superieure
        qui encadreront la recherche dans l-index qui contiend les vecteurs/paragraphes
        """
                
        nombreElements = self.count (index)
        nombreElements_voulu = int ((proportion * nombreElements)/100.)
        borne_inferieur_max = (nombreElements - nombreElements_voulu)
        
        borne_inferieur_random = randint (0, borne_inferieur_max)
        borne_superieur = borne_inferieur_random + nombreElements_voulu
        # en proportion
        borne_inferieur_random = int((borne_inferieur_random*self.dispersion)/nombreElements)
        borne_superieur = int((borne_superieur*self.dispersion)/nombreElements)
        return borne_inferieur_random, borne_superieur
        
        
    
    def search_par_bloc_vecteur(self,
                                 index,
                                 isVecteur = True,
                                 isParagraphe = False,
                                 ID_random_min = 0,
                                 ID_random_max = 0,
                                 isRandom = True,

                                 isID = False,
                                 ID_min = "",
                                 ID_max  = "",
                                 ID_sort  = 'asc',
                                 


                                 size = 1000 ,

                                ) :
        
        
        _source = ['ID',]
        if isVecteur: 
            _source .append( 'vecteur')
        if isParagraphe :
            _source.append('paragraphe')
            
                
        
        listeMust = []
        listeSort = []
        
             
        
        if isRandom :
                  
            listeMust.append({ "range" : {
                                    "ID_random" : {
                                        "gte" :ID_random_min,
                                        "lt" : ID_random_max,
                                                    },
                                    },
                              })
                    
        if isID :
            rangeID = { "range" : {
                                    "ID" : {
                                        "gte" :ID_min,
                                        "lte" : ID_max,
                                                    },
                                    },
                              }
            
            
            listeMust.append (rangeID)
            
            
        
        if not ID_sort is None :
            sort = {'ID' : {'order' : ID_sort},}
            listeSort.append(sort)
        
        
        
        query = {'_source': _source,
                      "query": {'bool': {'must': listeMust,
                                        },},
                                                    
                       "sort" : listeSort,
                    }
                        
                    

            
        
        
        
        
        #print (index, " et ",query, 'pour size =', size)
        res = self.es.search (index = index, body = query , size = size)
        hits = res['hits']
        nombre= hits['total'] ['value']
        vrai_hits= hits['hits']
                   
        return nombre, vrai_hits
    
    
    
    def search_par_bloc (self,
                         index,
                         
                         ID,                     # inused Mais compatibilite oblige
                         ID_reference_min,
                         ID_reference_max ,
                         ID_reference_sort = 'asc',
                         isReference = True,
                         
                         isID = False,
                         ID_min = "",
                         ID_max  = "",
                         ID_sort  = 'asc',
                         
                         isVariable = False,
                         nom_variableQuery = None,
                         variable_min = "",
                         variable_max = "",
                         variable_sort = 'asc',
                         _source = None,
                         
                         size = 10000 ,
                         
                        ) :
        
        #modif ################################################
        # warning toujours encadre par ID reference (à mettre au pas.....)
        #         ascendant ou descendant est une propriete de chaque variable
        listeMust = []
        listeSort = []
        
        
        
        
        if isReference :
            taille = len (self.ID_reference_base)
            ID_reference_min_string  = (self.ID_reference_base + str(ID_reference_min) ) [-taille:]
            ID_reference_max_string  = (self.ID_reference_base + str(ID_reference_max) ) [-taille:]
            
            listeMust.append({ "range" : {
                                    "ID_reference" : {
                                        "gte" :ID_reference_min_string,
                                        "lt" : ID_reference_max_string,
                                                    },
                                    },
                              })
        
        if not ID_reference_sort is None :
            sort = {'ID_reference' : {'order' : ID_reference_sort},}
            listeSort.append(sort)
            
        
        
        
        if isVariable :
            
            
            rangeVariable = { "range" : {
                                    nom_variableQuery : {
                                                    "gte" :variable_min,
                                                    "lt" : variable_max,
                                                                },
                                    },
                              }
            listeMust.append (rangeVariable)
            
        if not variable_sort is None :
            
            sort = {nom_variableQuery : {'order' : variable_sort},}
            listeSort.append(sort)
            
            
            
            
                
        if isID :
                           
            rangeID = { "range" : {
                                    "ID" : {
                                        "gte" :ID_min,
                                        "lte" : ID_max,
                                                    },
                                    },
                              }
            
            
            listeMust.append (rangeID)
            
            
        
        if not ID_sort is None :
            sort = {'ID' : {'order' : ID_sort},}
            listeSort.append(sort)
        
        
        if _source is None :
            query = {"query": {'bool': {'must': listeMust,
                                        },},
                  "sort" : listeSort,
                    }
        else :
            query = {'query' : query,
                     '_source' : _source}
        
            
        
        
        
        
        #print('dans la classe, query =', query)
        
        res = self.es.search (index= index, body = query , size = size)
        hits = res['hits']
        nombre= hits['total'] ['value']
        vrai_hits= hits['hits']
        return nombre, vrai_hits 
    
    def tri (self, hits) :
        liste = []
        i = 0
        for data in hits :
            x = data ['_source'] ['ID_reference']
            liste.append([i, x])
            i += 1
            continue

        liste_trie = sorted (liste, key = lambda d  : (d[1], d[0]) )

        liste_hits_trie = []
        for numero, _ in liste_trie :
            data = hits [numero]
            liste_hits_trie.append(data)
            continue

        return liste_hits_trie
        
    
    
    def close (self,) :
        
        self.es.indices.refresh()       
        self.es.close()
        
        return
    
      

    



In [3]:

#%save test_kernel_vecteur.py 83

In [3]:
from random import randint
from datetime import datetime

from Entree_sortie_lock import Entree_sortie_lock
#from Kernel_BE import Kernel

def iterator (nombre) :
    vecteurG = [1.]*300
    paragrapheG = ['essai']*1000
    for i in range(0, nombre ):
        doc = {'ID' : i ,
               'vecteur' : vecteurG,
               'ID_random' : randint (1, 1000000),
               'paragraphe' : paragrapheG }
        #print (doc ['ID'])
        yield doc
        
def test_vecteur () :
    
    systeme = {  
    "createur" : "generic", 
    "date_emetteur" : str(datetime.now()),
    "http_auth" : None,
    "timeout" : 10,
    "host" : "localhost", 
    "port" : 9200 ,
    "zipChoisi" : 'bz2',
    "index_log_error" : "trace1",
    "index_log_warning" : "trace2",
    "index_log_trace" : "trace3",
    "index_system" : "systeme",
    "isPurge_existing_index_log" : True,
    "extra_elasticsearch_args" : None,
    "trace" : False,
    "ID_reference_base" : "000000000000000000000000000000",


        }

    k = Kernel (systeme)
    nombre_vecteur = 11
    docs = iterator (nombre_vecteur)
    index = 'test'
    isPurge_existing_index = True
    resultat = k.bulk_vecteur (docs,
                                   index = index,
                                  isPurge_existing_index = isPurge_existing_index,
                                  chunk_size = 2000,
                                 )
    
    assert (resultat == True)

    index = 'test'
    
    #print ("premiere lecture")
    #print ('index =', index)
    nombre, vrai_hits = k.search_par_bloc_vecteur(
                                                     index,
                                                     isRandom = False,

                                                     isID = False,
                                                     ID_min = "0",
                                                     ID_max  = str(nombre_vecteur),
                                                     ID_sort  = 'asc',
                                                     size = 1000 ,
                                                )
    
    
    #print ("recu premiere lecture = \n",len(vrai_hits) )
    #print ("recu premiere lecture = \n", vrai_hits [0])
    assert ( nombre == nombre_vecteur ), 'erreur nombre de bulk'

    #print ()
    #print ("seconde lecture")
    #print ('index =', index)
    nombre, vrai_hits = k.search_par_bloc_vecteur(
                                                     index,
                                                     isRandom = False,
                                                     isID = True,
                                                     ID_min = 0,
                                                     ID_max  = 9,
                                                     ID_sort  = 'asc',
                                                     size = 1000 ,

                                                    )
    
    #print ("recu seconde lecture = \n",len(vrai_hits) )
    #print ("recu seconde lecture = \n", vrai_hits [0])
    assert nombre == 10  , ' erreur sur search vecteur :' + str(nombre)

if __name__ == '__main__' :
    test_vecteur ()
    print ('vecteur OK')

vecteur OK


In [20]:
#%save test_search_par_bloc.py 18

In [18]:
import random, time, copy, json
from datetime import datetime



import sys
path = "../outils"
if path not in sys.path : 
    sys.path.append (path)
from Kernel_BE import Kernel

from pprint import PrettyPrinter 
def P (stuff , pp = PrettyPrinter(indent=4)) :
    return pp.pprint(stuff)


def test_search_par_bloc() :
    arg = {}

    arg ['pathDico_evenements'] = '../data/dico_evenements_2.txt'
    arg ['pathDico_systeme'] = '../data/dico_systeme_2.txt'
    #  variable pour alimentation bloc

    arg ['ID_reference_min'] = 0
    arg ['ID_reference_max'] = 100000000
    arg ['ID_reference_sort'] = 'asc'
    arg ['isReference'] = False






    arg ['isVariable'] =  True
    arg ['nom_variableQuery'] = "date_evenement"
    arg ['variable_min'] = '2021-02-01 00:00:00'
    arg ['variable_max'] = '2021-02-04 00:00:00'
    arg ['variable_sort'] = 'asc'

    arg ['isID'] = True
    arg ['ID_min'] = "couple_cadre_1"
    arg ['ID_max']  = "couple_cadre_sup_1"
    arg ['ID_sort']  = None
    arg ['isTrace'] = True


    systeme = {  
              "createur" : "generic", 
              "date_emetteur" : str(datetime.now()),
              "http_auth" : None,
              "timeout" : 10,
              "host" : "localhost", 
              "port" : 9200 ,
              "zipChoisi" : 'bz2',
              "index_log_error" : "trace1",
              "index_log_warning" : "trace2",
              "index_log_trace" : "trace3",
              "index_system" : "systeme",
              "isPurge_existing_index_log" : True,
              "extra_elasticsearch_args" : None,
              "trace" : False,
              "ID_reference_base" : "000000000000000000000000000000",


                    }


    kernel = Kernel(systeme)

    debut_bloc = arg['ID_reference_min']
    fin_bloc = arg['ID_reference_max']
    ID_reference_sort = arg ['ID_reference_sort']
    isReference = arg ['isReference']



    isID = arg['isID']
    ID_min = arg ['ID_min']
    ID_max  = arg ['ID_max']
    ID_sort  = arg ['ID_sort']

    isVariable =  arg ['isVariable']
    nom_variableQuery = arg ['nom_variableQuery']
    variable_min = arg ['variable_min']
    variable_max = arg ['variable_max']
    variable_sort = arg ['variable_sort']


    isTrace = arg ['isTrace']
    
    
    size = 700
    taille_lu, hits = kernel.search_par_bloc ("data_fake_",
                                               " ", #deprecated
                                               debut_bloc,
                                               fin_bloc,
                                               size = size,
                                               isReference = isReference,
                                               ID_reference_sort = ID_reference_sort,

                                               isID = isID,
                                               ID_min = ID_min,
                                               ID_max  = ID_max ,
                                               ID_sort  = ID_sort,

                                               isVariable = isVariable,
                                               nom_variableQuery = nom_variableQuery,
                                               variable_min = variable_min,
                                               variable_max = variable_max,
                                               variable_sort = variable_sort,
                                             )


    assert taille_lu == 154, 'erreur sur search par bloc' 
    
    arg = {}
    nom_environnement = 'test'
    arg ['nom_environnement'] =  'nom_environnement'
    
    
    pathFile_evenements = '/dico_evenements_2.json'
    pathFile_systeme = '/dico_systeme_2.json'

    arg_entree_sortie_lock = {} 
    arg_entree_sortie_lock ['nom_environnement'] = nom_environnement
    arg_entree_sortie_lock['pathFile'] = pathFile_evenements
    Entree_sortie_evenements = Entree_sortie_lock (arg_entree_sortie_lock)
    dico_evenements, etat = Entree_sortie_evenements.lire()
    arg ['pathDico_evenements'] = dico_evenements
    #P (dico_evenements)

    arg_entree_sortie_lock ['pathFile'] = pathFile_systeme
    Entree_sortie_systeme = Entree_sortie_lock (arg_entree_sortie_lock)
    dico_systeme, etat = Entree_sortie_systeme.lire()
    arg ['pathDico_systeme'] = dico_systeme
    
    
    
    #  variable pour alimentation bloc

    arg ['ID_reference_min'] = 0
    arg ['ID_reference_max'] = 100000000
    arg ['ID_reference_sort'] = None
    arg ['isReference'] = False






    arg ['isVariable'] = True
    arg ['nom_variableQuery'] = "date_evenement"
    arg ['variable_min'] = '2021-02-01 00:00:00'
    arg ['variable_max'] = '2021-02-09 00:00:00'
    arg ['variable_sort'] = None

    arg ['isID'] = False
    arg ['ID_min'] = "couple_cadre_0"
    arg ['ID_max']  = "homme_ouvrier_1"
    arg ['ID_sort']  = 'asc'
    
    arg ['isTrace'] = False
    
    debut_bloc = arg['ID_reference_min']
    fin_bloc = arg['ID_reference_max']
    ID_reference_sort = arg ['ID_reference_sort']
    isReference = arg ['isReference']



    isID = arg['isID']
    ID_min = arg ['ID_min']
    ID_max  = arg ['ID_max']
    ID_sort  = arg ['ID_sort']

    isVariable =  arg ['isVariable']
    nom_variableQuery = arg ['nom_variableQuery']
    variable_min = arg ['variable_min']
    variable_max = arg ['variable_max']
    variable_sort = arg ['variable_sort']
    

    isTrace = arg ['isTrace']
    
    
    size = 1344
    taille_lu, hits = kernel.search_par_bloc ("data_fake",
                                               " ", #deprecated
                                               debut_bloc,
                                               fin_bloc,
                                               size = size,
                                               isReference = isReference,
                                               ID_reference_sort = ID_reference_sort,

                                               isID = isID,
                                               ID_min = ID_min,
                                               ID_max  = ID_max ,
                                               ID_sort  = ID_sort,

                                               isVariable = isVariable,
                                               nom_variableQuery = nom_variableQuery,
                                               variable_min = variable_min,
                                               variable_max = variable_max,
                                               variable_sort = variable_sort,
                                             )
    
    
    
    liste_ID = []
    memoire = {}
    for hit in hits :
        enreg = hit ['_source']
        ID = enreg ['ID']
        
        if not ID in memoire :
            liste_ID.append (ID)
            memoire [ID] = None
        continue
        
    liste_ID_voulu = [ID for ID in memoire.keys()]
    liste_ID_voulu.sort ()
    
    assert liste_ID_voulu == liste_ID
    
    
    
    
        
    

if __name__ == '__main__' :
    test_search_par_bloc()
    print ('fin test search par bloc OK')

fin test search par bloc OK


In [7]:
#%save test_kernel_general.py 7

In [8]:
import json, time
import numpy as np
from datetime import datetime

from Kernel_BE import Kernel



        
        
def  test_kernel_general () :

    systeme = {  
              "createur" : "generic", 
              "date_emetteur" : str(datetime.now()),
              "http_auth" : None,
              "timeout" : 10,
              "host" : "localhost", 
              "port" : 9200 ,
              "zipChoisi" : 'bz2',
              "index_log_error" : "trace1",
              "index_log_warning" : "trace2",
              "index_log_trace" : "trace3",
              "index_system" : "systeme",
              "isPurge_existing_index_log" : True,
              "extra_elasticsearch_args" : None,
              "trace" : False,
              "ID_reference_base" : "000000000000000000000000000000",


                    }

    ID_reference_base = systeme ["ID_reference_base"]
    k = Kernel(systeme)
    index = "test"
    k.create_index (index)
    k.delete_index (index)




    array = np.array ([i for i in range(0, 10)])
    data = k.array2liste (array,)
    arrayF = k.liste2array( data,dtype = np.int32)

    assert array.any()  == arrayF.any() , 'array <=> faux'

    array = np.array ([float(i)*0.1 for i in range(0, 10)])
    data = k.array2liste (array,)
    arrayF = k.liste2array( data,dtype = np.float64)

    assert array.any()  == arrayF.any(), 'array <=> faux'
    #print (array)
    #print (arrayF)
    ID_reference_base = systeme ["ID_reference_base"]
    taille = len(ID_reference_base)
    def iterateur (kernel, nombre = 1000) :

        for i in range(0, nombre) :
            for j in range(0,10) :
                dico = {
                      "ID" : ("0000"+ str(i)) [-4:] ,
                      "date_evenement" : (ID_reference_base + str(i))[-taille:],

                      "ID_reference" : (ID_reference_base  +str(i))[-taille:],

                       "nom_variable" : "variable_"+ str(j),
                       "valeur" : 1000000000000,
                }
                yield dico

    nombre = 10000    
    docs = iterateur (k, nombre = nombre)
    index = "test"

    t = time.time()
    k.bulk ( docs, index, isPurge_existing_index = True,)
    #print ("bulk =",k.count (index) == nombre*10, " en temps =", time.time () - t)



    # on test les logs

    #auteur, etape,  message

    auteur = "test1"
    etape = "etape1"
    message = "message1"

    k.log_error (auteur, etape,  message)
    k.log_error (auteur, etape,  message)

    k.log_warning (auteur, etape,  message)
    k.log_warning (auteur, etape,  message)

    k.log_trace (auteur, etape,  message)
    k.log_trace (auteur, etape,  message)
    
    voulu = {'origine': 'error', 'auteur': 'test1', 'etape': 'etape1',
             'message': 'message1'}
    
    result = k.get_logs_error() 
    assert len (result) == 2, "log error"
    voulu ['origine'] = 'error'
    r = result [0]
    del r ['date']
    assert voulu == r, str(result [0])
    
    result = k.get_logs_warning()
    assert len (result) == 2, "log warning"
    voulu ['origine'] = 'warning'
    r = result [0]
    del r ['date']
    assert voulu == r, "log warning"
    
    result = k.get_logs_trace()
    assert len (result) == 2, "log trace"
    voulu ['origine'] = 'trace'
    r = result [0]
    del r ['date']
    assert voulu == r, "log trace"
    
    

    
    # test search bloc ######################

    j = 5

    i = 0
    pas = 1000 # donne 1000 ligne

    ID_reference_min = i
    ID_reference_max = i + pas
    ID = str(i+2)
    index = "test"
    # test avec un size ################


    time.time()
    taille, hits = k.search_par_bloc (index,
                             ID,                     # inused Mais compatibilite oblige
                             ID_reference_min,
                             ID_reference_max ,
                             ID_reference_sort = None,

                             isID = False,
                             ID_min = "",
                             ID_max  = "",
                             ID_sort  = 'asc',

                             isVariable = False,
                             nom_variableQuery = None,
                             variable_min = "",
                             variable_max = "",
                             variable_sort = None,

                             size = pas*10 ,
                                     )

    assert taille != 0  , 'erreur de lecture'
        


    







    assert taille == len(hits)  and taille == pas * 10 , 'erreur taille dans lecture de 10000 éléments'
        



    t = time.time()


    taille, hits = k.search_par_bloc (index,
                             ID,                     # inused Mais compatibilite oblige
                             ID_reference_min,
                             ID_reference_max ,
                             ID_reference_sort = None,

                             isID = False,
                             ID_min = "",
                             ID_max  = "",
                             ID_sort  = None,

                             isVariable = False,
                             nom_variableQuery = None,
                             variable_min = "",
                             variable_max = "",
                             variable_sort = None,

                             size = pas*10 ,
                                     )

    
    assert taille == len(hits)  and taille == 10000 , 'erreur taille dans lecture de 10000 éléments'
    



    r = hits [0]['_source'] ['ID_reference']

    taille_1 = len(ID_reference_base)
    voulu = (ID_reference_base  +str (ID_reference_min))[-taille_1:]

    assert r == voulu , 'erreur dans le contenu du premier hit'
        



    r = hits [taille - 1]['_source'] ['ID_reference']

    taille_1 = len(ID_reference_base)             
    voulu = (ID_reference_base  + str(ID_reference_max-1))[-taille_1:] # on calcul le ID_reference
    assert r == voulu , 'erreur dans le contenu du dernier hit'
    


    # avec ID voulu
    j = 5 # ID
    ID_min = ("0000"+ str(j)) [-4:] # il apparait une fois par ligne donc il a 10 variables
    ID_max = ("0000"+ str(j+1)) [-4:]


    i = 0
    pas = 100 # nombre de ligne


    ID_reference_min = i 
    ID_reference_max = i + pas
    index = "test"
    t = time.time()
    taille, hits = k.search_par_bloc (index,
                                     ID,                     # inused Mais compatibilite oblige

                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                      isReference = True,

                                     isID = True,
                                     ID_min = ID_min ,
                                     ID_max  = ID_max ,
                                     ID_sort  = 'asc',

                                     isVariable = False,
                                     nom_variableQuery = None,
                                     variable_min = "",
                                     variable_max = "",
                                     variable_sort = None,
                                      size = pas*10 ,
                                     )

    



    assert taille == len(hits) and taille == 20 , 'nombre d evenements pour 1 ID'
        





    ID_voulu =  ("0000"+ str(j)) [-4:]    # ID_min 
    taille_variable = 10
    liste_variable =  []
    for i in range(0, taille_variable ) :
        nom_variable = hits [i] ['_source'] ['nom_variable']
        assert hits [i] ['_source'] ['ID'] == ID_min
        
        liste_variable.append (nom_variable)
        continue

    voulu = ['variable_0', 'variable_1', 'variable_2', 'variable_3', 'variable_4', 'variable_5',
             'variable_6', 'variable_7', 'variable_8', 'variable_9']

    assert liste_variable == voulu , 'variable non trie par ID_reference'
        


    # on verifie que ID_reference est trie

    ID_reference_courant = ""
    for i in range(0, 10) :
        ID_reference = hits[i] ['_source'] ['ID_reference']
        assert ID_reference >= ID_reference_courant
        
        ID_reference_courant = ID_reference
        continue

    # avec ID voulu mais non ID dans ID_reference
    j = 5 # ID
    ID_min = ("0000"+ str(j)) [-4:] # il apparait une fois par ligne donc il a 10 variables
    ID_max = ("0000"+ str(j+1)) [-4:]


    i = 0
    pas = 5


    ID_reference_min = i 
    ID_reference_max = i + pas
    index = "test"
    t = time.time()
    taille, hits = k.search_par_bloc (index,
                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = True,

                                     isID = True,
                                     ID_min = ID_min ,
                                     ID_max  = ID_max ,
                                     ID_sort  = 'asc',

                                     isVariable = False,
                                     nom_variableQuery = None,
                                     variable_min = "",
                                     variable_max = "",
                                     variable_sort = None,

                                     size = pas*10 ,)


    assert  taille == 0 , 'on  load un ID sur une plage ou il n est pas'
        

    # on croise date et ID_ref

    j = 5 # ID
    ID_min = ("0000"+ str(j)) [-4:] # il apparait une fois par ligne donc il a 10 variables
    ID_max = ("0000"+ str(j+1)) [-4:]


    i = 0
    pas = 5


    ID_reference_min = i 
    ID_reference_max = i + pas

    variable_min = 0
    variable_max = 2

    taille = len(ID_reference_base)
    variable_min_string = (ID_reference_base + str(variable_min))[-taille:]
    variable_max_string = (ID_reference_base + str(variable_max))[-taille:]

    index = "test"
    t = time.time()
    taille, hits = k.search_par_bloc (index,
                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = True,

                                     isID = True,
                                     ID_min = ID_min ,
                                     ID_max  = ID_max ,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = variable_min_string,
                                     variable_max = variable_max_string,
                                     variable_sort = 'asc',

                                     size = pas*10 ,)


    assert taille == 0 , 'reference 0, 5 et ID 5, 6 et date_evenement 0, 2 taille ?'
        



    j = 5 # ID
    ID_min = ("0000"+ str(j)) [-4:] # il apparait une fois par ligne donc il a 10 variables
    ID_max = ("0000"+ str(j+1)) [-4:]




    i = 0
    pas = 5

    ID_reference_min = i 
    ID_reference_max = i + pas

    variable_min = 0
    variable_max = 2

    taille = len(ID_reference_base)
    variable_min_string = (ID_reference_base + str(variable_min))[-taille:]
    variable_max_string = (ID_reference_base + str(variable_max))[-taille:]

    index = "test"
    t = time.time()
    taille, hits = k.search_par_bloc (index,
                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'desc',
                                     isReference = True,

                                     isID = False,
                                     ID_min = ID_min ,
                                     ID_max  = ID_max ,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = variable_min_string,
                                     variable_max = variable_max_string,
                                     variable_sort = 'desc',

                                     size = pas*10 ,)


    assert taille == 20 , 'ID_reference et date_evenement ne correspondent pas taille'
        
    liste_date_evenement = []
    liste_ID_reference = []

    for hit in hits :
        enreg = hit ['_source']
        ID_reference = enreg ['ID_reference']
        liste_ID_reference.append(ID_reference)

        date_evenement = enreg ['date_evenement']
        liste_date_evenement.append(date_evenement) 

        continue
    isFirst = True    
    for date, ID_reference in zip (liste_date_evenement, liste_ID_reference ) :
        if isFirst :
            date_courant, ID_reference_courant = date, ID_reference
            isFirst = False
        assert date  <= date_courant and ID_reference <= ID_reference_courant
        
        continue


    

    

    # test de search par date

    index = "test"
    t = time.time()

    variable_min = 1
    variable_max = 21

    taille = len(ID_reference_base)
    variable_min_string = (ID_reference_base + str(variable_min))[-taille:]
    variable_max_string = (ID_reference_base + str(variable_max))[-taille:]

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = False,
                                     ID_min = "",
                                     ID_max  = "",
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = variable_min_string,
                                     variable_max = variable_max_string,
                                     variable_sort = 'asc',

                                     size = pas*10 ,)


    assert taille == 200 , 'erreur dans search sur variable date_evenement'
    

    isFirst = True    
    for hit in hits :
        enreg = hit ['_source']
        date = enreg ['date_evenement']
        if isFirst :
            assert date == '000000000000000000000000000001'
            
            date_courante = date
            isFirst = False
            continue
        assert date >= date_courante , 'erreur de sens non asc'
        
        date_courante = date
        continue


    # test search par bloc temps X ID
    taille = len(ID_reference_base)
    # min
    i = 0
    ID_min = ("0000"+ str(i)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]

    ID_reference_min = (ID_reference_base  +str(i))[-taille:],

    #max
    i = 10
    ID_max = ("0000"+ str(i)) [-4:] 
    date_evenement_max = (ID_reference_base + str(i))[-taille:]

    ID_reference_max = (ID_reference_base  +str(i))[-taille:]






    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = pas*10 ,)




    assert taille == 100 , 'erreur de taille dans la lecture par bloc date X ID'
    
    ID_reference_base = systeme ["ID_reference_base"]
    taille = len(ID_reference_base)
    isFirst = True
    numero_ligne = 0
    for hit in hits :
        doc = hit ['_source']
        i = int(numero_ligne/10)
        j = numero_ligne % 10
        dico = {
                      "ID" : ("0000"+ str(i)) [-4:] ,
                      "date_evenement" : (ID_reference_base + str(i))[-taille:],

                      "ID_reference" : (ID_reference_base  +str(i))[-taille:],

                       "nom_variable" : "variable_"+ str(j),
                       "valeur" : 1000000000000,
                }
        assert dico == doc , 'erreur de contenu dans le search bloc date x ID'
            
        numero_ligne += 1
        continue

    
    # test search par bloc ID_reference X ID
    taille = len(ID_reference_base)
    # min
    i = 0
    ID_min = ("0000"+ str(i)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]

    ID_reference_min = (ID_reference_base  +str(i))[-taille:],

    #max
    i = 10
    ID_max = ("0000"+ str(i)) [-4:] 
    date_evenement_max = (ID_reference_base + str(i))[-taille:]

    ID_reference_max = (ID_reference_base  +str(i))[-taille:]






    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = True,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = False,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = pas*10 ,)





    assert taille == 100 , 'erreur de taille dans la lecture par bloc date X ID'
        

    ID_reference_base = systeme ["ID_reference_base"]
    taille = len(ID_reference_base)
    isFirst = True
    numero_ligne = 0
    for hit in hits :
        doc = hit ['_source']
        i = int(numero_ligne/10)
        j = numero_ligne % 10
        dico = {
                      "ID" : ("0000"+ str(i)) [-4:] ,
                      "date_evenement" : (ID_reference_base + str(i))[-taille:],

                      "ID_reference" : (ID_reference_base  +str(i))[-taille:],

                       "nom_variable" : "variable_"+ str(j),
                       "valeur" : 1000000000000,
                }
        assert  dico == doc , 'erreur de contenu dans le search bloc date x ID'
            
        numero_ligne += 1
        continue


    # test search par bloc temps X en limitant ID
    taille = len(ID_reference_base)
    # min
    i = 0
    i_ID = 2
    ID_min = ("0000"+ str(i_ID)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]

    ID_reference_min = (ID_reference_base  +str(i))[-taille:],

    #max
    i = 10
    ID_max = ("0000"+ str(i_ID + 1 )) [-4:] 
    date_evenement_max = (ID_reference_base + str(i))[-taille:]

    ID_reference_max = (ID_reference_base  +str(i))[-taille:]






    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = pas*10 ,)





    assert taille == 20 , 'erreur de taille dans la lecture par bloc date X ID avec  limitation de ID'
        

    ID_reference_base = systeme ["ID_reference_base"]
    taille = len(ID_reference_base)
    isFirst = True
    numero_ligne = 0
    ID_cherche = ("0000"+ str(i_ID)) [-4:]
    ID_cherche_suivant  = ("0000"+ str(i_ID+1)) [-4:]
    for hit in hits :
        doc = hit ['_source']
        if doc ['ID']  != ID_cherche and doc ['ID']  != ID_cherche_suivant:
            print ('erreur de contenu dans le search bloc date x ID avec  limitation de ID')
            print (" voulu =", ID_cherche)
            print()
            print (' lu =', doc ['ID'])
            raise ValueError
        numero_ligne += 1
        continue



    # on test le count

    taille = k.count(index)

    assert taille == 100000 , 'erreur du count sur index lu='
        


    #  on essaye le count by search


    

    taille = len(ID_reference_base)
    # min
    i = 1
    i_ID = 2
    ID_min = ("0000"+ str(i_ID)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]



    #max
    i = 1
    i_ID = 2
    inc = 20
    ID_max = ("0000"+ str(i_ID + inc )) [-4:] 
    date_evenement_max = (ID_reference_base + str(i+inc))[-taille:]

    ID_reference_min = 0

    ID_reference_max = 1000

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = True,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = 100 ,

                                     )

    




    taille = len(ID_reference_base)
    # min
    i = 1
    i_ID = 2
    ID_min = ("0000"+ str(i_ID)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]



    #max
    i = 1
    i_ID = 2
    inc = 20
    ID_max = ("0000"+ str(i_ID + inc )) [-4:] 
    date_evenement_max = (ID_reference_base + str(i+inc))[-taille:]


    ID_reference_min = 12
    
    ID_reference_max = 1000

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = True,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = 100 ,

                                     )


    



    taille = len(ID_reference_base)
    # min
    i = 1
    i_ID = 2
    ID_min = ("0000"+ str(i_ID)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]



    #max
    i = 1
    i_ID = 2
    inc = 20
    ID_max = ("0000"+ str(i_ID + inc )) [-4:] 
    date_evenement_max = (ID_reference_base + str(i+inc))[-taille:]


    ID_reference_min = 0
    #print ('ID_reference_min =', ID_reference_min)
    ID_reference_max = 1000

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = True,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort = None,

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = None,

                                     size =100 ,

                                     )


    



    #### ##########
    taille = len(ID_reference_base)
    # min
    i = 1
    i_ID = 2
    ID_min = ("0000"+ str(i_ID)) [-4:] 
    date_evenement_min = (ID_reference_base + str(i))[-taille:]



    #max
    i = 1
    i_ID = 2
    inc = 20
    ID_max = ("0000"+ str(i_ID + inc )) [-4:] 
    date_evenement_max = (ID_reference_base + str(i+inc))[-taille:]

    ID_reference_min = 0

    ID_reference_max = 1000

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort = None,

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = None,

                                     size =100 ,

                                     )
    
    arg = {}
    arg ['isTrace'] = False
    
    
    #  variable pour alimentation bloc_date

    arg ['ID_reference_min'] = 0
    arg ['ID_reference_max'] = 100000000
    arg ['ID_reference_sort'] = 'asc'
    arg ['isReference'] = False






    arg ['isVariable'] =  True
    arg ['nom_variableQuery'] = "date_evenement"
    arg ['variable_min'] = '2021-02-01 00:00:00.000'
    arg ['variable_max'] = '2021-02-10 00:00:00.000'
    arg ['variable_sort'] = 'asc'

    arg ['isID'] = False
    arg ['ID_min'] = "couple_cadre_1"
    arg ['ID_max']  = "couple_cadre_sup_1"
    arg ['ID_sort']  = 'asc'
    
    arg ['isTrace'] = False

    
    
    
    
    arg = {}
    arg ['isTrace'] = False
    
    
    #  variable pour alimentation bloc_date

    arg ['ID_reference_min'] = 0
    arg ['ID_reference_max'] = 100000000
    arg ['ID_reference_sort'] = 'asc'
    arg ['isReference'] = False






    arg ['isVariable'] =  True
    arg ['nom_variableQuery'] = "date_evenement"
    arg ['variable_min'] = '2021-02-01 00:00:00.000'
    arg ['variable_max'] = '2021-02-10 00:00:00.000'
    arg ['variable_sort'] = 'asc'

    arg ['isID'] = False
    arg ['ID_min'] = "couple_cadre_1"
    arg ['ID_max']  = "couple_cadre_sup_1"
    arg ['ID_sort']  = 'asc'
    
    arg ['isTrace'] = False

    

    

    # on avance en prenant la derniere date 


    hit = hits [len(hits) - 1]
    enreg = hit ['_source']
    date_evenement_min = enreg ['date_evenement']


    taille = len(ID_reference_base)
    # min
    i = 1
    i_ID = 2
    ID_min = ("0000"+ str(i_ID)) [-4:] 
    hit = hits [len(hits) - 1]
    enreg = hit ['_source']
    date_evenement_min_int = int(enreg ['date_evenement'])
    date_evenement_min = (ID_reference_base + str(date_evenement_min_int + 1))[-taille:]



    #max
    i = 1
    i_ID = 2
    inc = 20
    ID_max = ("0000"+ str(i_ID + inc )) [-4:] 
    date_evenement_max = (ID_reference_base + str(i+inc))[-taille:]

    ID_reference_min = 0

    ID_reference_max = 1000000

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = 100 ,

                                     )

    


    hit = hits [len(hits) - 1]
    enreg = hit ['_source']
    date_evenement_min = enreg ['date_evenement']

    

    ######## data_fake

    index = 'data_fake_'

    # min

    ID_min = "couple_cadre_1"
    date_evenement_min = '2021-02-01 00:00:00'

    #     "couple_cadre_sup_0", "couple_cadre_sup_1"               
    #max

    ID_max = "couple_cadre_sup_1"
    date_evenement_max = '2021-02-20 00:00:00'

    ID_reference_min = 0
    ID_reference_max = 1000000

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = True,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = 'asc',

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort = 'asc',

                                     size = 100 ,

                                     )

    



    hit = hits [len(hits) - 1]
    enreg = hit ['_source']

    date_evenement_min = enreg ['date_evenement']
    #v = convertir ( date_evenement_min )
    r = date_evenement_min.split ('.')
    date_evenement_min = r[0]
    dateCourante = datetime.strptime(date_evenement_min, "%Y-%m-%d %M:%S:%f")
    timestamp = datetime.timestamp(dateCourante)
    timestamp +=1
    date_evenement_min = datetime.fromtimestamp(timestamp)

    taille, hits = k.search_par_bloc (index,

                                     ID,                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = 'asc',
                                     isReference = False,

                                     isID = False,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = None,

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = date_evenement_min,
                                     variable_max = date_evenement_max,
                                     variable_sort  = None,

                                     size = 100 ,

                                     )


    


    hit = hits [0]
    enreg = hit ['_source']
    date_evenementcourant = enreg ['date_evenement']

    for hit in hits :
        enreg = hit ['_source']
        date_evenement = enreg ['date_evenement']
        ID = enreg ['ID']
        variable = enreg ['nom_variable'] 





    isFirst = True
    for hit in hits :
        if isFirst :
            enreg = hit ['_source']
            ID_reference = enreg ['ID_reference']
            courant = ID_reference
            continue

        enreg = hit ['_source']
        ID_reference = enreg ['ID_reference']
        
        assert courant <= ID_reference , 'erreur de tri sur ID_reference' 
            
        courant= enreg ['ID_reference']
        continue

    


if __name__ == '__main__' :
    test_kernel_general ()
    print ('Fin du test')


        
   

Fin du test


In [9]:
#%save test_alimentation_bloc_date.py 11

In [10]:
#### import json, time
import numpy as np
from datetime import datetime

from Kernel_BE import Kernel


def test_alimentation_bloc_date ():
    
    
    
    #  variable pour alimentation bloc_date
    
    systeme = {  
    "createur" : "generic", 
    "date_emetteur" : str(datetime.now()),
    "http_auth" : None,
    "timeout" : 10,
    "host" : "localhost", 
    "port" : 9200 ,
    "zipChoisi" : 'bz2',
    "index_log_error" : "trace1",
    "index_log_warning" : "trace2",
    "index_log_trace" : "trace3",
    "index_system" : "systeme",
    "isPurge_existing_index_log" : True,
    "extra_elasticsearch_args" : None,
    "trace" : False,
    "ID_reference_base" : "000000000000000000000000000000",


        }
    

    k = Kernel (systeme)
    
    index = 'data_fake'

    ID_reference_min = 0
    ID_reference_max = 100000000
    ID_reference_sort = 'asc'
    isReference = False






    isVariable =  True
    nom_variableQuery = "date_evenement"
    variable_min = '2021-02-01 00:00:00.000'
    variable_max = '2021-03-10 00:00:00.000'
    variable_sort = 'asc'

    isID = False
    ID_min = "couple_cadre_1"
    ID_max = "couple_cadre_sup_1"
    ID_sort = 'asc'
    
    isTrace = False
    size = 192*7
    
    taille, hits = k.search_par_bloc (index,

                                     'ID',                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = None,
                                     isReference = False,

                                     isID = False,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = None,

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = variable_min,
                                     variable_max = variable_max,
                                     variable_sort  = 'asc',

                                     size = size ,

                                     )
    
    assert taille == size, "erreur lecture globale sur date"
    
    #print ("dans lecture avec size =", size, " on a len(hist) =",len(hits), " et taille =", taille)
    taille_totale_lu = len(hits)
    memoire = {}
    date_max = ""
    isFirst = True
    for hit in hits :
        enreg = hit ['_source']
        date_evenement = enreg ['date_evenement']
        if date_evenement > date_max :
            date_max = date_evenement
        
        if isFirst :
            taille_evenement = len(date_evenement)
            isFirst = False
        
        if taille_evenement != len(date_evenement) :
            print('### erreur possible pour date_evenement =',date_evenement) 
            raise ValueError
        try :
            memoire [date_evenement] += 1
        except :
            memoire [date_evenement] = 1
        continue
    
    
        
    #print ('apres analyse de la premiere lecture, on a taille_evenement =',taille_evenement)
    #print ('apres analyse de la premiere lecture, on a len (memoire) =',len(memoire))
    #print ('apres analyse de la premiere lecture, on a date_max =',date_max)
    #print ()
                
    
    size = 700
    
    
    taille, hits = k.search_par_bloc (index,

                                     'ID',                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = None,
                                     isReference = False,

                                     isID = False,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = None,

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = variable_min,
                                     variable_max = variable_max,
                                     variable_sort  = 'asc',

                                     size = size ,

                                     )
    
    #print ("dans lecture avec size =", size, " on a len(hist) =",len(hits), " et taille =", taille)
    #print ()
    taille_deja_lu = len(hits)
    
    memoire = {}
    date_max = ""
    for hit in hits :
        enreg = hit ['_source']
        date_evenement = enreg ['date_evenement']
        if date_evenement > date_max :
            date_max = date_evenement
            
        try :
            memoire [date_evenement] += 1
        except :
            memoire [date_evenement] = 1
        continue
    
    #print ('apres analyse de la deuxieme lecture, on a len (memoire) =',len(memoire))
    #print ('apres analyse de la deuxieme lecture, on a date_max (la plus grande) =',date_max)
    #print ()
    hit = hits [len(hits)- 1]
    enreg = hit ['_source']
    date_evenement_min = enreg ['date_evenement']
    
    
    format_standard = '%Y-%m-%d %H:%M:%S.%f'
    dateCourante = datetime.strptime(date_evenement_min, format_standard)
    timestamp = datetime.timestamp(dateCourante)
    timestamp += 1
    date_evenement_min = str(datetime.fromtimestamp(timestamp))
    
    
    #print ('preparation deuxieme lecture')
    #print ('variable min calculée dans alimentation_bloc =',date_evenement_min)
    #print ( 'et variable_max =' , variable_max)
    
    variable_min = date_evenement_min
    
    taille, hits = k.search_par_bloc (index,

                                     'ID',                     # inused Mais compatibilite oblige
                                     ID_reference_min,
                                     ID_reference_max ,
                                     ID_reference_sort = None,
                                     isReference = False,

                                     isID = False,
                                     ID_min = ID_min,
                                     ID_max  = ID_max,
                                     ID_sort  = None,

                                     isVariable = True,
                                     nom_variableQuery = "date_evenement",
                                     variable_min = variable_min,
                                     variable_max = variable_max,
                                     variable_sort  = 'asc',

                                     size = size ,

                                     )
    #print ("dans lecture bloc final  avec size =", size, " on a len(hist) =",len(hits), " et taille =", taille)
    #print ('bloc final len(hits) =', len(hits))
    hit = hits [len(hits)- 1]
    enreg = hit ['_source']
    date_evenement_min = enreg ['date_evenement']
    #print ('date_evenement finale apres la dernier lecture =', date_evenement_min)
    
    assert len(hits) + taille_deja_lu == taille_totale_lu, 'erreur de, lecture sur date'
    
if __name__ == '__main__' :
    test_alimentation_bloc_date ()
    print ('fin test_alimentation_bloc_date')    

    


fin test_alimentation_bloc_date
